In [1]:
import pandas as pd
import numpy as np
import psycopg2
import pickle
import math

In [2]:
## Connect to SQL database
conn = psycopg2.connect(database = 'cuzegotk',user = 'cuzegotk',password = 'NekW2BqJ8hW1wO3hCdpuEESPiP-y131V',host = 'raja.db.elephantsql.com')
cur = conn.cursor()
## Get Joined table of the company financials, and ratings
cur.execute('SELECT companyfinancials.ticker, revenue, revenue_ly, rating FROM companyfinancials JOIN companyratings ON companyfinancials.ticker = companyratings.ticker')
financials_a = cur.fetchall()
cur.close()
conn.close()

In [3]:
## Connect to SQL database
conn = psycopg2.connect(database = 'cuzegotk',user = 'cuzegotk',password = 'NekW2BqJ8hW1wO3hCdpuEESPiP-y131V',host = 'raja.db.elephantsql.com')
cur = conn.cursor()
## Get Joined table of the company balance sheets, quarter-ends, and shares_outstanding
cur.execute('SELECT Q.ticker, sector, shares_outstanding, total_assets, total_liabilities, intangible_assets, shareholders_equity, total_equity,revenue,gross_profit,operating_profit, net_income FROM companybalancesheets B JOIN companyquarterends Q ON Q.ticker = B.ticker JOIN sharesoutstanding S ON S.ticker = B.ticker JOIN companylisting L ON L.ticker = S.ticker')
financials_q = cur.fetchall()
cur.close()
conn.close()

In [4]:
maximum = pd.read_csv('./datasets/featurescalingmax.csv', usecols=[1])
maximum.index = ['industrials', 'consumer_discretionary', 'tech', 'health_care', 'energy', 'consumer_staples', 'utilities', 'real_estate', 'materials', 'communication_services', 'eps', 'bvs', 'roe', 'roa', 'debt-to-equity', 'debt-to-asset', 'gross-margin', 'operating-margin', 'net-margin', 'yoy-revenue-growth']
minimum = pd.read_csv('./datasets/featurescalingmin.csv', usecols=[1])
minimum.index = ['industrials', 'consumer_discretionary', 'tech', 'health_care', 'energy', 'consumer_staples', 'utilities', 'real_estate', 'materials', 'communication_services', 'eps', 'bvs', 'roe', 'roa', 'debt-to-equity', 'debt-to-asset', 'gross-margin', 'operating-margin', 'net-margin', 'yoy-revenue-growth']

In [5]:
maximum = maximum.iloc[:,0].values
minimum = minimum.iloc[:,0].values

In [6]:
# Define columns annual financials
a_columns = ['ticker', 'revenue', 'revenue_ly', 'rating']
# Define columns of quarter financials
q_columns = ['ticker', 'sector', 'shares_outstanding', 'total_assets', 'total_liabilities', 'intangible_assets', 'shareholders_equity', 'total_equity', 'revenue', 'gross_profit', 'operating_profit', 'net_income']

In [7]:
financials_a = pd.DataFrame(financials_a, columns=a_columns)
financials_q = pd.DataFrame(financials_q, columns=q_columns)

In [8]:
# Check Missing values
financials_a[financials_a['rating']=='No Ratings']

,ticker,revenue,revenue_ly,rating
85,BCRH,3.540000e+07,4.940000e+07,No Ratings
111,BXEFF,2.049230e+08,2.254380e+08,No Ratings
205,DIIBF,2.619513e+09,2.577668e+09,No Ratings
474,NM,5.177390e+08,4.630490e+08,No Ratings
555,RIBT,1.476200e+07,1.335500e+07,No Ratings
642,TOO,1.416424e+09,1.110284e+09,No Ratings
673,UTI,3.169650e+08,3.242630e+08,No Ratings
720,XRF,9.277700e+07,5.698300e+07,No Ratings


In [9]:
# Manually research data and filling in data
financials_a.iloc[85, 3] = 'Strong Sell'
financials_a.iloc[111, 3] = 'Moderate Sell'
financials_a.iloc[205, 3] = 'Moderate Buy'
financials_a.iloc[474, 3] = 'Strong Sell'
financials_a.iloc[555, 3] = 'Strong Buy'
financials_a.iloc[642, 3] = 'Hold'
financials_a.iloc[673, 3] = 'Strong Sell'
financials_a.iloc[720, 3] = 'Hold'

In [10]:
ratings = financials_a['rating']
financials_a = financials_a.iloc[:,:-1]

In [11]:
ratings.value_counts()

Moderate Buy     341
Hold             185
Strong Buy       102
Moderate Sell     73
Strong Sell       27
Name: rating, dtype: int64

In [12]:
encode_ratings = {
    'Strong Sell': 0,
    'Moderate Sell': 0,
    'Hold': 1,
    'Moderate Buy': 2,
    'Strong Buy': 2
}

In [13]:
ratings = ratings.map(encode_ratings)

In [14]:
financials_q[financials_q['sector'] == 'nan']

,ticker,sector,shares_outstanding,total_assets,total_liabilities,intangible_assets,shareholders_equity,total_equity,revenue,gross_profit,operating_profit,net_income
250,EVRZF,nan,451340000.0,9.480932e+09,7.270352e+09,1.049878e+09,1.964960e+09,2.210580e+09,NaN,NaN,NaN,NaN
429,MDOMF,nan,46790000.0,9.001381e+08,1.988873e+08,8.297885e+07,6.447466e+08,7.012508e+08,6.062000e+09,2.469000e+09,1.025000e+09,0.0


In [15]:
financials_q.iloc[250,1] = 'Materials'
financials_q.iloc[429,1] = 'Consumer Discretionary'

In [16]:
## Encoding categorical date using Pandas get_dummies
X1 = pd.get_dummies(financials_q['sector']).values

In [17]:
## Feature for measuring a company's earnings and its margin of safety, EPS and Book Value per share
X2 = (financials_q.net_income / financials_q.shares_outstanding).values.reshape(len(X1), 1)
X3 = ((financials_q.total_assets - financials_q.total_liabilities - financials_q.intangible_assets) / financials_q.shares_outstanding).values.reshape(len(X1), 1)

In [18]:
## Measure of company's profitability, Return on Equity and Return on Assets
X4 = (financials_q.net_income / financials_q.shareholders_equity).values.reshape(len(X1), 1)
X5 = (financials_q.net_income / financials_q.shareholders_equity).values.reshape(len(X1), 1)

In [19]:
## Measure of company's financial leverage and liquidity, Debt-to-Equity ratio and Debt-to-Asset ratio
X6 = (financials_q.total_liabilities / financials_q.shareholders_equity).values.reshape(len(X1), 1)
X7 = (financials_q.total_liabilities / financials_q.total_assets).values.reshape(len(X1), 1)

In [20]:
## Measure of company's margins, gross, operating, and net
X8 = (financials_q.gross_profit / financials_q.revenue).values.reshape(len(X1), 1)
X9 = (financials_q.operating_profit / financials_q.revenue).values.reshape(len(X1), 1)
X10 = (financials_q.net_income / financials_q.revenue).values.reshape(len(X1), 1)

In [21]:
## Measure of year-over-year growth in sales
X11 = ((financials_a.revenue - financials_a.revenue_ly) / financials_a.revenue_ly).values.reshape(len(X1), 1)

In [22]:
## Set of features we will use for our machine learning algorithm
features = np.concatenate((X1[:,1:], X2, X3, X4, X5, X6, X7, X8, X9, X10, X11), axis = 1)

In [23]:
df = pd.DataFrame(features)

In [24]:
feature_scaling = (features - minimum) / (maximum - minimum)

In [25]:
filename = 'model.pkl'
model = pickle.load(open(filename, 'rb'))

In [26]:
predicted_ratings = []
for i in range(len(feature_scaling)):
    try:
        predicted_ratings.append(int(model.predict(feature_scaling[i,:].reshape(1,-1))[0]))
    except:
        predicted_ratings.append(np.nan)

In [27]:
for i,v in enumerate(predicted_ratings):
    if v == 0:
        predicted_ratings[i] = 'Sell'
    elif v == 1:
        predicted_ratings[i] = 'Hold'
    elif v ==2:
        predicted_ratings[i] = 'Buy'
    else:
        predicted_ratings[i] = 'No Ratings'
        
for i,v in enumerate(ratings):
    if v == 0:
        ratings[i] = 'Sell'
    elif v == 1:
        ratings[i] = 'Hold'
    elif v == 2:
        ratings[i] = 'Buy'
    else:
        ratings[i] = 'No Ratings'

In [28]:
model_predictions = []
for i in range(len(financials_a)):
    model_predictions.append([financials_a.iloc[i,0], ratings.values[i], predicted_ratings[i]])

In [29]:
## Connect to SQL database
conn = psycopg2.connect(database = 'cuzegotk',user = 'cuzegotk',password = 'NekW2BqJ8hW1wO3hCdpuEESPiP-y131V',host = 'raja.db.elephantsql.com')
cur = conn.cursor()
## Transfer CSV to SQL Database
for i in range(0,len(model_predictions)):
    cur.execute('INSERT INTO modelpredictions VALUES (%s, %s, %s)', tuple(model_predictions[i]))
    conn.commit()
cur.close()
conn.close()